In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pyspark

# Team 10:
# Bùi Tấn Lộc
# Hà Huy Quốc
# Lê Trần Bảo Nam

In [ ]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark.version

In [ ]:
movies=spark.read.options(header='True',inferschema='True',delimiter=',').csv("/kaggle/input/msis405l21cttt-spring-2021-assignment-7/movies_small.csv")
movies.printSchema()
movies.show()

In [ ]:
import pyspark.sql.functions as f
from pyspark.sql.functions import array_contains
from pyspark.sql.functions import col

In [ ]:
#1. Show ID, title, and genres of all movies with ‘Action’ included in their genres
movies.withColumn("genres_array",f.split(f.col("genres"),"\|")).filter(f.array_contains(f.col('genres_array'),'Action')).select('movieId','title','genres').show()
                                                               

In [ ]:
#2. Show ID, title, and the number of genres of each movie, sorted by movieId
movies.withColumn("genres_array",f.split(f.col("genres"),"\|")).withColumn("count",f.size('genres_array')).select('movieId','title','count').sort('movieId').show()

In [ ]:
#3. Show the number of movies of each genre, sorted by the count number

movies.withColumn('genres_array', f.split('genres', '\|')).withColumn('genres', f.explode('genres_array')).groupBy('genres').count().sort('count').show(5)

In [ ]:
#4. Show the list of all movies associated with each genres.
movies.withColumn('genres_array', f.split('genres', '\|')).withColumn('genres', f.explode('genres_array')).groupBy('genres').agg(f.collect_list("title").alias('moives_list')).show(truncate=100)

In [ ]:
#5. Show the years of the first_appearance of ‘Animation’ and ‘Sci-Fi’ movies
movies.withColumn('arr', f.split('title', ' ')).withColumn('last_element', f.element_at('arr', -1)).withColumn('year', f.substring('last_element', 2,4).cast('int')).withColumn('arr_genres', f.split('genres', '\|')).withColumn('genres', f.explode('arr_genres')).groupBy('genres').agg(f.min('year').alias('first_appearance')).filter(f.substring('genres',0,10).isin(['Sci-Fi','Animation'])).select(f.col('genres').alias('genre'), 'first_appearance').show(1000, truncate=False)